In [1]:
%use klaxon
%use lets-plot

import java.net.*
import jetbrains.letsPlot.scale.*


In [2]:
data class FftSample(
    val index: Long, 
    val binCount: Int, 
    val samplesCount: Int, 
    val sampleRate: Float, 
    val magnitude: List<Double>,
    val phase: List<Double>,
    val frequency: List<Double>,
    val time: Long
)
data class Result(val offset: Long, val value: FftSample)

val k = Klaxon()

val data = URL("http://localhost:6800/table/fft/last?interval=30ms").openStream().reader().readLines()
.map { k.parse<Result>(it)!! }

In [3]:
val freqCutOff = 10 to 3000 // Hz
val table = data.asSequence().map {v -> 
        v.value.magnitude.asSequence()
            .zip(v.value.frequency.asSequence())
            .filter { it.second > freqCutOff.first && it.second < freqCutOff.second}
            .map { arrayOf(v.value.time / 1_000_000.0, it.second, it.first) }
    }
    .flatten()
    .toList()

val dataFrame = mapOf(
    "time" to table.map { it[0] },
    "freq" to table.map { it[1] },
    "value" to table.map { it[2] }
)


In [6]:

lets_plot(dataFrame) {x = "time"; y = "freq"; fill = "value"} + 
    ggsize(1000, 300) + 
    geom_tile() + 
    scale_fill_gradient(low = "light_green", high = "red")
    